In [7]:
import requests
from bs4 import BeautifulSoup
import re
import json
from datetime import datetime, timedelta
import pandas as pd
import warnings
import time
import math
import os
from dotenv import load_dotenv
warnings.filterwarnings('ignore')

load_dotenv('config.env')

class ComprehensiveSGDUSDScrapers:
    def __init__(self, fred_api_key=None):
        self.fred_api_key = fred_api_key
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
        }

    def scrape_bond_yield(self, url, bond_name):
        try:
            response = requests.get(url, headers=self.headers, timeout=15)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            yield_selectors = [
                '[data-test="instrument-price-last"]',
                '.text-2xl',
                '.instrument-price_last__KQzyA',
                '.last-price-value',
                '.text-5xl',
                '.pid-23705-last',
                '[data-symbol] [data-field="regularMarketPrice"]'
            ]
            
            for selector in yield_selectors:
                element = soup.select_one(selector)
                if element:
                    text = element.get_text(strip=True)
                    yield_match = re.search(r'\d+\.\d{2,3}', text)
                    if yield_match:
                        potential_yield = float(yield_match.group())
                        if 0.1 <= potential_yield <= 15.0:
                            return potential_yield
            
            main_price_elements = soup.find_all(['span', 'div'], 
                class_=re.compile(r'(price|last|current|main|big|large)', re.I))
            
            for element in main_price_elements:
                text = element.get_text(strip=True)
                if re.search(r'\b\d+\.\d{2,3}%?\b', text):
                    match = re.search(r'\d+\.\d{2,3}', text)
                    if match:
                        potential_yield = float(match.group())
                        if 0.1 <= potential_yield <= 15.0:
                            return potential_yield
            
            for element in soup.find_all(['td', 'th', 'span', 'div', 'strong']):
                text = element.get_text(strip=True)
                pattern = r'^\d+\.\d{2,3}%?$'
                if re.match(pattern, text):
                    numeric_value = float(re.sub(r'[^\d.]', '', text))
                    if 0.1 <= numeric_value <= 15.0:
                        return numeric_value
            
            return None
            
        except Exception as e:
            print(f"Error scraping {bond_name}: {e}")
            return None

    def scrape_percentage_change(self, url, name, min_range=-30, max_range=30):
        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            change_selectors = [
                '[data-test="instrument-price-change-percent"]',
                '.text-red-500', '.text-green-500',
                '.changePercent', '.change-percent'
            ]
            
            for selector in change_selectors:
                element = soup.select_one(selector)
                if element:
                    text = element.get_text(strip=True)
                    pct_match = re.search(r'([+-]?\d+\.\d+)%', text)
                    if pct_match:
                        change_pct = float(pct_match.group(1))
                        if min_range <= change_pct <= max_range:
                            return change_pct
            
            for element in soup.find_all(['span', 'div'], class_=re.compile(r'change|pct', re.I)):
                text = element.get_text(strip=True)
                pct_match = re.search(r'([+-]?\d+\.\d+)%', text)
                if pct_match:
                    change_pct = float(pct_match.group(1))
                    if min_range <= change_pct <= max_range:
                        return change_pct
            
            for row in soup.find_all('tr'):
                cells = row.find_all(['td', 'th'])
                if len(cells) >= 2:
                    label = cells[0].get_text(strip=True).lower()
                    value = cells[1].get_text(strip=True)
                    if 'change' in label and '%' in value:
                        pct_match = re.search(r'([+-]?\d+\.\d+)%', value)
                        if pct_match:
                            change_pct = float(pct_match.group(1))
                            if min_range <= change_pct <= max_range:
                                return change_pct
            
            return None
            
        except Exception as e:
            print(f"Error scraping {name} change: {e}")
            return None

    def scrape_marketwatch_change_pct(self, url, name, min_range=-20, max_range=20):
        try:
            response = requests.get(url, headers=self.headers, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            
            percent_elements = soup.find_all(class_=re.compile(r'percent', re.I))
            for element in percent_elements:
                text = element.get_text(strip=True)
                if '%' in text:
                    pct_match = re.search(r'([+-]?\d+\.\d+)%', text)
                    if pct_match:
                        change_pct = float(pct_match.group(1))
                        if min_range <= change_pct <= max_range:
                            return change_pct
            
            change_selectors = [
                '[class*="change"]',
                '[class*="Change"]', 
                '.pct-change',
                '.price-change',
                '[data-module*="change"]'
            ]
            
            for selector in change_selectors:
                elements = soup.select(selector)
                for element in elements:
                    text = element.get_text(strip=True)
                    if '%' in text:
                        pct_match = re.search(r'([+-]?\d+\.\d+)%', text)
                        if pct_match:
                            change_pct = float(pct_match.group(1))
                            if min_range <= change_pct <= max_range:
                                return change_pct
            
            tables = soup.find_all('table')
            for table in tables:
                rows = table.find_all('tr')
                for row in rows:
                    cells = row.find_all(['td', 'th'])
                    for cell in cells:
                        text = cell.get_text(strip=True)
                        if '%' in text and len(text) < 20:
                            pct_match = re.search(r'([+-]?\d+\.\d+)%', text)
                            if pct_match:
                                change_pct = float(pct_match.group(1))
                                if min_range <= change_pct <= max_range:
                                    return change_pct
            
            for element in soup.find_all(['span', 'div']):
                text = element.get_text(strip=True)
                if re.match(r'^[+-]?\d+\.\d+%$', text):
                    pct_match = re.search(r'([+-]?\d+\.\d+)', text)
                    if pct_match:
                        change_pct = float(pct_match.group(1))
                        if min_range <= change_pct <= max_range:
                            return change_pct
            
            return None
            
        except Exception as e:
            print(f"Error scraping {name} change from MarketWatch: {e}")
            return None

    def scrape_single_value(self, url, name, min_val, max_val):
        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            price_selectors = [
                '[data-test="instrument-price-last"]',
                '.text-2xl', '.text-5xl',
                '.instrument-price_last__KQzyA',
                '.last-price-value'
            ]
            
            for selector in price_selectors:
                element = soup.select_one(selector)
                if element:
                    text = element.get_text(strip=True)
                    clean_text = re.sub(r'[,\s]', '', text)
                    match = re.search(r'\d+\.?\d*', clean_text)
                    if match:
                        value = float(match.group())
                        if min_val <= value <= max_val:
                            return value
            
            for element in soup.find_all(['span', 'div'], class_=re.compile(r'price|last|current', re.I)):
                text = element.get_text(strip=True)
                clean_text = re.sub(r'[,\s]', '', text)
                match = re.search(r'\d+\.?\d*', clean_text)
                if match:
                    value = float(match.group())
                    if min_val <= value <= max_val:
                        return value
            
            return None
            
        except Exception as e:
            print(f"Error scraping {name}: {e}")
            return None

    def scrape_single_value_targeted(self, url, name, min_val, max_val):
        try:
            response = requests.get(url, headers=self.headers, timeout=15)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            element = soup.find('div', {'data-test': 'instrument-price-last'})
            if element:
                text = element.get_text(strip=True)
                clean_text = text.replace(',', '')
                if re.match(r'^\d{4}\.\d{2}$', clean_text):
                    value = float(clean_text)
                    if min_val <= value <= max_val:
                        return value
            
            return self.scrape_single_value(url, name, min_val, max_val)
            
        except Exception as e:
            print(f"Error scraping {name}: {e}")
            return None

    def scrape_marketwatch_value(self, url, name, min_val, max_val):
        try:
            response = requests.get(url, headers=self.headers, timeout=15)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            percent_elements = soup.find_all(class_=re.compile(r'percent', re.I))
            for element in percent_elements:
                text = element.get_text(strip=True)
                clean_text = re.sub(r'[,\s]', '', text)
                match = re.search(r'\d+\.?\d*', clean_text)
                if match:
                    value = float(match.group())
                    if min_val <= value <= max_val:
                        return value
            
            change_selectors = [
                '[class*="change"]',
                '[class*="Change"]', 
                '.pct-change',
                '.price-change',
                '[class*="price"]',
                '[class*="last"]',
                '[class*="current"]',
                'span.value'
            ]
            
            for selector in change_selectors:
                elements = soup.select(selector)
                for element in elements:
                    text = element.get_text(strip=True)
                    clean_text = re.sub(r'[,\s]', '', text)
                    match = re.search(r'\d+\.?\d*', clean_text)
                    if match:
                        value = float(match.group())
                        if min_val <= value <= max_val:
                            return value
            
            tables = soup.find_all('table')
            for table in tables:
                rows = table.find_all('tr')
                for row in rows:
                    cells = row.find_all(['td', 'th'])
                    for cell in cells:
                        text = cell.get_text(strip=True)
                        clean_text = re.sub(r'[,\s]', '', text)
                        match = re.search(r'\d+\.?\d*', clean_text)
                        if match:
                            value = float(match.group())
                            if min_val <= value <= max_val:
                                return value
            
            for element in soup.find_all(['span', 'div']):
                text = element.get_text(strip=True)
                clean_text = re.sub(r'[,\s]', '', text)
                
                if name == "STI":
                    if re.match(r'^\d{4}\.\d{2}$', clean_text):
                        value = float(clean_text)
                        if min_val <= value <= max_val:
                            return value
                elif "stock" in name.lower():
                    if re.match(r'^\d{2}\.\d{2}$', clean_text):
                        value = float(clean_text)
                        if min_val <= value <= max_val:
                            return value
                else:
                    match = re.search(r'\d+\.?\d*', clean_text)
                    if match:
                        value = float(match.group())
                        if min_val <= value <= max_val:
                            return value
            
            return None
            
        except Exception as e:
            print(f"Error scraping {name} from MarketWatch: {e}")
            return None

    def scrape_fred_data(self, series_id, limit=1):
        if not self.fred_api_key:
            return None
            
        try:
            url = f"https://api.stlouisfed.org/fred/series/observations"
            params = {
                'series_id': series_id,
                'api_key': self.fred_api_key,
                'file_type': 'json',
                'limit': limit,
                'sort_order': 'desc'
            }
            
            response = requests.get(url, params=params, timeout=10)
            data = response.json()
            
            if 'observations' in data and data['observations']:
                latest = data['observations'][0]
                return {
                    'value': float(latest['value']) if latest['value'] != '.' else None,
                    'date': latest['date']
                }
            return None
            
        except Exception as e:
            print(f"Error getting FRED data for {series_id}: {e}")
            return None

    def scrape_singapore_1y_bond(self):
        url = 'https://www.investing.com/rates-bonds/singapore-1-year-bond-yield'
        return self.scrape_bond_yield(url, "Singapore 1Y")

    def scrape_singapore_2y_bond(self):
        url = 'https://www.investing.com/rates-bonds/singapore-2-year-bond-yield'
        return self.scrape_bond_yield(url, "Singapore 2Y")

    def scrape_singapore_5y_bond(self):
        url = 'https://www.investing.com/rates-bonds/singapore-5-year-bond-yield'
        return self.scrape_bond_yield(url, "Singapore 5Y")

    def scrape_singapore_10y_bond(self):
        url = 'https://www.investing.com/rates-bonds/singapore-10-year-bond-yield'
        return self.scrape_bond_yield(url, "Singapore 10Y")

    def scrape_us_2y_treasury(self):
        url = 'https://www.investing.com/rates-bonds/u.s.-2-year-bond-yield'
        return self.scrape_bond_yield(url, "US 2Y")

    def scrape_us_5y_treasury(self):
        try:
            url = 'https://www.investing.com/rates-bonds/u.s.-5-year-bond-yield'
            response = requests.get(url, headers=self.headers, timeout=15)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            yield_selectors = [
                '[data-test="instrument-price-last"]',
                '.text-2xl',
                '.instrument-price_last__KQzyA',
                '.last-price-value',
                '.text-5xl'
            ]
            
            for selector in yield_selectors:
                element = soup.select_one(selector)
                if element:
                    text = element.get_text(strip=True)
                    yield_match = re.search(r'\d+\.\d{2,3}', text)
                    if yield_match:
                        potential_yield = float(yield_match.group())
                        if 0.1 <= potential_yield <= 15.0:
                            return potential_yield
            
            return None
            
        except Exception as e:
            print(f"Error scraping US 5Y yield: {e}")
            return None

    def scrape_us_10y_treasury(self):
        url = 'https://www.investing.com/rates-bonds/u.s.-10-year-bond-yield'
        return self.scrape_bond_yield(url, "US 10Y")

    def scrape_us_30y_treasury(self):
        url = 'https://www.investing.com/rates-bonds/u.s.-30-year-bond-yield'
        return self.scrape_bond_yield(url, "US 30Y")

    def scrape_sgd_usd_current_rate(self):
        try:
            url = "https://www.investing.com/currencies/sgd-usd"
            response = requests.get(url, headers=self.headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            for element in soup.find_all(['span', 'div'], class_=re.compile(r'price|last|current', re.I)):
                text = element.get_text(strip=True)
                pattern = r'^0\.\d{4}$'
                if re.match(pattern, text):
                    rate_val = float(text)
                    if 0.65 <= rate_val <= 0.85:
                        return rate_val
            
            url = "https://tradingeconomics.com/singapore/currency"
            response = requests.get(url, headers=self.headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            for element in soup.find_all(['span', 'div', 'td']):
                text = element.get_text(strip=True)
                if re.search(r'1\.[0-9]{4}', text):
                    rate_match = re.search(r'1\.\d{4}', text)
                    if rate_match:
                        rate = float(rate_match.group())
                        if 1.2 <= rate <= 1.5:
                            return 1 / rate
            
            return None
            
        except Exception as e:
            print(f"Error getting SGD/USD rate: {e}")
            return None

    def scrape_sgd_usd_daily_change_pct(self):
        url = "https://www.investing.com/currencies/sgd-usd"
        return self.scrape_percentage_change(url, "SGD/USD daily change", -10, 10)

    def scrape_sgd_usd_weekly_change_pct(self):
        try:
            url = "https://finance.yahoo.com/quote/SGDUSD=X/"
            response = requests.get(url, headers=self.headers, timeout=15)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            for table in soup.find_all('table'):
                for row in table.find_all('tr'):
                    cells = row.find_all(['td', 'th'])
                    if len(cells) >= 2:
                        label = cells[0].get_text(strip=True).lower()
                        value = cells[1].get_text(strip=True)
                        
                        if any(term in label for term in ['week', '1w', '7d']) and '%' in value:
                            pct_match = re.search(r'([+-]?\d+\.\d+)%', value)
                            if pct_match:
                                change_pct = float(pct_match.group(1))
                                if -20 <= change_pct <= 20:
                                    return change_pct
            
            for element in soup.find_all(['span', 'div']):
                text = element.get_text(strip=True)
                weekly_pattern = r'(week|1w|7d)[^%]*([+-]?\d+\.\d+)%'
                match = re.search(weekly_pattern, text, re.IGNORECASE)
                if match:
                    change_pct = float(match.group(2))
                    if -20 <= change_pct <= 20:
                        return change_pct
            
            return None
            
        except Exception as e:
            print(f"Error scraping SGD/USD weekly change: {e}")
            return None

    def scrape_sgd_usd_monthly_change_pct(self):
        try:
            url = "https://finance.yahoo.com/quote/SGDUSD=X/"
            response = requests.get(url, headers=self.headers, timeout=15)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            for table in soup.find_all('table'):
                for row in table.find_all('tr'):
                    cells = row.find_all(['td', 'th'])
                    if len(cells) >= 2:
                        label = cells[0].get_text(strip=True).lower()
                        value = cells[1].get_text(strip=True)
                        
                        if any(term in label for term in ['month', '1m', '30d']) and '%' in value:
                            pct_match = re.search(r'([+-]?\d+\.\d+)%', value)
                            if pct_match:
                                change_pct = float(pct_match.group(1))
                                if -30 <= change_pct <= 30:
                                    return change_pct
            
            for element in soup.find_all(['span', 'div']):
                text = element.get_text(strip=True)
                monthly_pattern = r'(month|1m|30d)[^%]*([+-]?\d+\.\d+)%'
                match = re.search(monthly_pattern, text, re.IGNORECASE)
                if match:
                    change_pct = float(match.group(2))
                    if -30 <= change_pct <= 30:
                        return change_pct
            
            try:
                url = "https://www.investing.com/currencies/sgd-usd"
                response = requests.get(url, headers=self.headers, timeout=10)
                soup = BeautifulSoup(response.content, 'html.parser')
                
                for row in soup.find_all('tr'):
                    cells = row.find_all(['td', 'th'])
                    if len(cells) >= 2:
                        label = cells[0].get_text(strip=True).lower()
                        value = cells[1].get_text(strip=True)
                        
                        if any(term in label for term in ['month', '1m', '30d']) and '%' in value:
                            pct_match = re.search(r'([+-]?\d+\.\d+)%', value)
                            if pct_match:
                                change_pct = float(pct_match.group(1))
                                if -30 <= change_pct <= 30:
                                    return change_pct
            except:
                pass
            
            try:
                url = "https://tradingeconomics.com/sgdusd:cur"
                response = requests.get(url, headers=self.headers, timeout=10)
                soup = BeautifulSoup(response.content, 'html.parser')
                
                for element in soup.find_all(['span', 'div', 'td']):
                    text = element.get_text(strip=True)
                    if 'month' in text.lower() and '%' in text:
                        pct_match = re.search(r'([+-]?\d+\.\d+)%', text)
                        if pct_match:
                            change_pct = float(pct_match.group(1))
                            if -30 <= change_pct <= 30:
                                return change_pct
            except:
                pass
            
            return None
            
        except Exception as e:
            print(f"Error scraping SGD/USD monthly change: {e}")
            return None

    def scrape_sgd_usd_52w_high(self):
        try:
            url = "https://www.investing.com/currencies/sgd-usd"
            response = requests.get(url, headers=self.headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            for row in soup.find_all('tr'):
                cells = row.find_all(['td', 'th'])
                if len(cells) >= 2:
                    label = cells[0].get_text(strip=True).lower()
                    value = cells[1].get_text(strip=True)
                    
                    if any(term in label for term in ['52', 'week', 'wk']) and 'range' in label:
                        range_match = re.findall(r'0\.\d{4}', value)
                        if len(range_match) == 2:
                            return max(float(x) for x in range_match)
                    
                    elif any(term in label for term in ['52', 'week', 'wk']) and 'high' in label:
                        rate_match = re.search(r'0\.\d{4}', value)
                        if rate_match:
                            return float(rate_match.group())
            
            for element in soup.find_all(['span', 'div']):
                if '52' in element.get_text():
                    parent = element.find_parent()
                    if parent:
                        rates = re.findall(r'0\.\d{4}', parent.get_text())
                        if len(rates) >= 2:
                            return max(float(x) for x in rates)
            
            return None
            
        except Exception as e:
            print(f"Error scraping SGD/USD 52w high: {e}")
            return None

    def scrape_sgd_usd_52w_low(self):
        try:
            url = "https://www.investing.com/currencies/sgd-usd"
            response = requests.get(url, headers=self.headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            for row in soup.find_all('tr'):
                cells = row.find_all(['td', 'th'])
                if len(cells) >= 2:
                    label = cells[0].get_text(strip=True).lower()
                    value = cells[1].get_text(strip=True)
                    
                    if any(term in label for term in ['52', 'week', 'wk']) and 'range' in label:
                        range_match = re.findall(r'0\.\d{4}', value)
                        if len(range_match) == 2:
                            valid_rates = [float(x) for x in range_match if 0.6 <= float(x) <= 0.9]
                            if len(valid_rates) == 2:
                                return min(valid_rates)
                    
                    elif any(term in label for term in ['52', 'week', 'wk']) and 'low' in label:
                        rate_match = re.search(r'0\.\d{4}', value)
                        if rate_match:
                            rate_val = float(rate_match.group())
                            if 0.6 <= rate_val <= 0.9:
                                return rate_val
            
            for element in soup.find_all(['span', 'div']):
                if '52' in element.get_text():
                    parent = element.find_parent()
                    if parent:
                        rates = re.findall(r'0\.\d{4}', parent.get_text())
                        valid_rates = [float(x) for x in rates if 0.6 <= float(x) <= 0.9]
                        if len(valid_rates) >= 2:
                            return min(valid_rates)
            
            return None
            
        except Exception as e:
            print(f"Error scraping SGD/USD 52w low: {e}")
            return None

    def get_comprehensive_forex_data(self):
        try:
            url = "https://www.investing.com/currencies/sgd-usd"
            response = requests.get(url, headers=self.headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            current_rate = None
            day_high = None
            day_low = None
            change_pct = None
            
            for row in soup.find_all('tr'):
                cells = row.find_all(['td', 'th'])
                if len(cells) >= 2:
                    label = cells[0].get_text(strip=True).lower()
                    value = cells[1].get_text(strip=True)
                    
                    if 'high' in label and ('day' in label or 'daily' in label):
                        if re.search(r'0\.\d{4}', value):
                            day_high = float(re.search(r'0\.\d{4}', value).group())
                    elif 'low' in label and ('day' in label or 'daily' in label):
                        if re.search(r'0\.\d{4}', value):
                            day_low = float(re.search(r'0\.\d{4}', value).group())
                    elif 'change' in label and '%' in value:
                        pct_match = re.search(r'([+-]?\d+\.\d+)%', value)
                        if pct_match:
                            change_pct = float(pct_match.group(1))
            
            if not current_rate:
                current_rate = self.scrape_sgd_usd_current_rate()
            
            return {
                'current_rate': current_rate,
                '24h_high': day_high,
                '24h_low': day_low,
                'daily_change_pct': change_pct
            }
            
        except Exception as e:
            print(f"Error getting comprehensive forex data: {e}")
            return {'current_rate': None, '24h_high': None, '24h_low': None, 'daily_change_pct': None}

    def scrape_eur_usd_rate(self):
        url = "https://www.investing.com/currencies/eur-usd"
        return self.scrape_single_value(url, "EUR/USD", 0.9, 1.3)

    def scrape_usd_jpy_rate(self):
        url = "https://www.investing.com/currencies/usd-jpy"
        return self.scrape_single_value(url, "USD/JPY", 100, 160)

    def scrape_eur_usd_change_pct(self):
        url = "https://www.investing.com/currencies/eur-usd"
        return self.scrape_percentage_change(url, "EUR/USD change", -10, 10)

    def scrape_usd_jpy_change_pct(self):
        url = "https://www.investing.com/currencies/usd-jpy"
        return self.scrape_percentage_change(url, "USD/JPY change", -10, 10)

    def scrape_vix(self):
        url = "https://www.investing.com/indices/volatility-s-p-500"
        return self.scrape_single_value(url, "VIX", 5, 100)

    def scrape_vix_level(self):
        url = "https://www.investing.com/indices/volatility-s-p-500"
        return self.scrape_single_value(url, "VIX", 5, 100)

    def scrape_vix_change_pct(self):
        url = "https://www.investing.com/indices/volatility-s-p-500"
        return self.scrape_percentage_change(url, "VIX change", -50, 50)

    def scrape_dxy(self):
        url = "https://www.investing.com/currencies/us-dollar-index"
        return self.scrape_single_value(url, "DXY", 70, 130)

    def scrape_dxy_level(self):
        url = "https://www.investing.com/currencies/us-dollar-index"
        return self.scrape_single_value(url, "DXY", 70, 130)

    def scrape_dxy_change_pct(self):
        url = "https://www.investing.com/currencies/us-dollar-index"
        return self.scrape_percentage_change(url, "DXY change", -10, 10)

    def scrape_gold_price(self):
        try:
            url = "https://www.investing.com/commodities/gold"
            response = requests.get(url, headers=self.headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            price_selectors = [
                '[data-test="instrument-price-last"]',
                '.text-2xl', '.text-5xl',
                '.instrument-price_last__KQzyA',
                '.last-price-value',
                'span[class*="text-"]',
                'div[class*="text-"]'
            ]
            
            for selector in price_selectors:
                elements = soup.select(selector)
                for element in elements:
                    text = element.get_text(strip=True)
                    clean_text = re.sub(r'[,\s]', '', text)
                    pattern = r'\b(\d{4})\.\d{2}\b'
                    match = re.search(pattern, clean_text)
                    if match:
                        value = float(clean_text)
                        if 1500 <= value <= 5000:
                            return value
            
            for element in soup.find_all(['span', 'div']):
                text = element.get_text(strip=True)
                pattern = r'^\d{4}\.\d{2}$'
                if re.match(pattern, text):
                    value = float(text)
                    if 1500 <= value <= 5000:
                        return value
            
            return None
            
        except Exception as e:
            print(f"Error scraping Gold price: {e}")
            return None

    def scrape_gold_change_pct(self):
        url = "https://www.investing.com/commodities/gold"
        return self.scrape_percentage_change(url, "Gold change", -10, 10)

    def scrape_oil_wti_price(self):
        url = "https://www.investing.com/commodities/crude-oil"
        return self.scrape_single_value(url, "Oil", 30, 150)

    def scrape_oil_wti_change_pct(self):
        url = "https://www.investing.com/commodities/crude-oil"
        return self.scrape_percentage_change(url, "Oil WTI change", -20, 20)

    def scrape_msci_asia_pacific(self):
        url = "https://www.investing.com/indices/msci-ac-asia-pacific"
        return self.scrape_single_value(url, "MSCI Asia", 100, 300)

    def scrape_sp500_level(self):
        url = "https://www.investing.com/indices/us-spx-500"
        return self.scrape_single_value_targeted(url, "S&P 500", 3000, 7000)

    def scrape_sp500_change_pct(self):
        url = "https://www.investing.com/indices/us-spx-500"
        return self.scrape_percentage_change(url, "S&P 500 change", -20, 20)

    def scrape_nikkei_level(self):
        url = "https://www.investing.com/indices/japan-ni225"
        return self.scrape_single_value(url, "Nikkei 225", 20000, 50000)

    def scrape_nikkei_change_pct(self):
        url = "https://www.investing.com/indices/japan-ni225"
        return self.scrape_percentage_change(url, "Nikkei change", -15, 15)

    def scrape_hang_seng_level(self):
        url = "https://www.investing.com/indices/hang-sen-40"
        return self.scrape_single_value(url, "Hang Seng", 15000, 35000)

    def scrape_hang_seng_change_pct(self):
        url = "https://www.investing.com/indices/hang-sen-40"
        return self.scrape_percentage_change(url, "Hang Seng change", -15, 15)

    def scrape_singapore_sti_level(self):
        url = "https://www.marketwatch.com/investing/index/sti?countrycode=sg"
        return self.scrape_marketwatch_value(url, "STI", 2500, 5000)

    def scrape_singapore_sti_change_pct(self):
        url = "https://www.investing.com/indices/ftse-singapore"
        return self.scrape_percentage_change(url, "STI change", -15, 15)

    def scrape_dbs_stock_price(self):
        url = "https://www.marketwatch.com/investing/stock/d05?countrycode=sg"
        return self.scrape_marketwatch_value(url, "DBS stock", 35, 65)

    def scrape_dbs_change_pct(self):
        url = "https://www.marketwatch.com/investing/stock/d05?countrycode=sg"
        return self.scrape_marketwatch_change_pct(url, "DBS change", -20, 20)

    def scrape_uob_stock_price(self):
        url = "https://www.marketwatch.com/investing/stock/u11?countrycode=sg"
        return self.scrape_marketwatch_value(url, "UOB stock", 20, 45)

    def scrape_uob_change_pct(self):
        try:
            url = "https://www.marketwatch.com/investing/stock/u11?countrycode=sg"
            response = requests.get(url, headers=self.headers, timeout=15)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            percent_elements = soup.find_all(class_=re.compile(r'percent', re.I))
            for element in percent_elements:
                text = element.get_text(strip=True)
                if '%' in text:
                    pct_match = re.search(r'([+-]?\d+\.\d+)%', text)
                    if pct_match:
                        change_pct = float(pct_match.group(1))
                        if -20 <= change_pct <= 20:
                            return change_pct
            
            change_selectors = [
                '[class*="change"]',
                '[class*="Change"]', 
                '.pct-change',
                '.price-change'
            ]
            
            for selector in change_selectors:
                elements = soup.select(selector)
                for element in elements:
                    text = element.get_text(strip=True)
                    if '%' in text:
                        pct_match = re.search(r'([+-]?\d+\.\d+)%', text)
                        if pct_match:
                            change_pct = float(pct_match.group(1))
                            if -20 <= change_pct <= 20:
                                return change_pct
            
            return None
            
        except Exception as e:
            print(f"Error scraping UOB change: {e}")
            return None

    def scrape_sg_reit_index(self):
        alternative_sources = [
            ("SPDR STI ETF", "https://www.investing.com/etfs/spdr-straits-times-index-etf"),
            ("iShares MSCI Singapore", "https://www.investing.com/etfs/ishares-msci-singapore"),
            ("Lion Phillip S REIT ETF", "https://www.investing.com/etfs/lion-phillip-s-reit-etf"),
            ("Mapletree REIT", "https://www.investing.com/equities/mapletree-commercial-trust")
        ]
        
        for source_name, url in alternative_sources:
            try:
                response = requests.get(url, headers=self.headers, timeout=10)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.content, 'html.parser')
                    
                    element = soup.find('div', {'data-test': 'instrument-price-last'})
                    if element:
                        text = element.get_text(strip=True)
                        clean_text = text.replace(',', '')
                        
                        if 'etf' in url.lower():
                            if re.match(r'^\d{1,3}\.\d{2}$', clean_text):
                                value = float(clean_text)
                                if 10 <= value <= 200:
                                    return value
                        else:
                            if re.match(r'^\d\.\d{2}$', clean_text):
                                value = float(clean_text)
                                if 0.5 <= value <= 5.0:
                                    return value
            except:
                continue
        
        return None

    def scrape_sg_reit_change_pct(self):
        try:
            reit_sources = [
                "https://www.investing.com/indices/ftse-epra-nareit-singapore-capped",
                "https://www.investing.com/indices/singapore-reit",
                "https://www.investing.com/etfs/ishares-asia-property-yield"
            ]
            
            for url in reit_sources:
                try:
                    response = requests.get(url, headers=self.headers, timeout=10)
                    if response.status_code == 200:
                        result = self.scrape_percentage_change(url, "Singapore REIT", -15, 15)
                        if result is not None:
                            return result
                except:
                    continue
            
            return None
            
        except Exception as e:
            print(f"Error scraping SG REIT change: {e}")
            return None

    def scrape_baltic_dry_index(self):
        try:
            url = "https://www.investing.com/indices/baltic-dry"
            return self.scrape_single_value(url, "Baltic Dry Index", 500, 5000)
        except Exception as e:
            print(f"Error scraping Baltic Dry Index: {e}")
            return None

    def scrape_baltic_dry_change_pct(self):
        try:
            url = "https://www.investing.com/indices/baltic-dry"
            response = requests.get(url, headers=self.headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            change_selectors = [
                '[data-test="instrument-price-change-percent"]',
                '.text-red-500', '.text-green-500',
                '[class*="change"]', '[class*="percent"]'
            ]
            
            for selector in change_selectors:
                elements = soup.select(selector)
                for element in elements:
                    text = element.get_text(strip=True)
                    pct_match = re.search(r'([+-]?\d+\.\d+)%', text)
                    if pct_match:
                        change_pct = float(pct_match.group(1))
                        if -20 <= change_pct <= 20:
                            return change_pct
            
            return None
            
        except Exception as e:
            print(f"Error scraping Baltic Dry change: {e}")
            return None

    def scrape_palm_oil_price(self):
        try:
            url = "https://www.investing.com/commodities/palm-oil"
            return self.scrape_single_value(url, "Palm Oil", 2000, 6000)
        except Exception as e:
            print(f"Error scraping Palm Oil price: {e}")
            return None

    def scrape_palm_oil_change_pct(self):
        try:
            url = "https://www.investing.com/commodities/palm-oil"
            response = requests.get(url, headers=self.headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            change_selectors = [
                '[data-test="instrument-price-change-percent"]',
                '.text-red-500', '.text-green-500',
                '[class*="change"]', '[class*="percent"]'
            ]
            
            for selector in change_selectors:
                elements = soup.select(selector)
                for element in elements:
                    text = element.get_text(strip=True)
                    pct_match = re.search(r'([+-]?\d+\.\d+)%', text)
                    if pct_match:
                        change_pct = float(pct_match.group(1))
                        if -20 <= change_pct <= 20:
                            return change_pct
            
            return None
            
        except Exception as e:
            print(f"Error scraping Palm Oil change: {e}")
            return None

    def scrape_sgd_usd_volatility_20d(self):
        try:
            return None
            
        except Exception as e:
            print(f"Error calculating SGD/USD 20d volatility: {e}")
            return None

    def scrape_sg_gdp_from_trading_economics(self):
        try:
            url = "https://tradingeconomics.com/singapore/gdp-growth-annual"
            response = requests.get(url, headers=self.headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            for element in soup.find_all(['span', 'div', 'td']):
                text = element.get_text(strip=True)
                pattern = r'^[+-]?\d+\.\d+%?$'
                if re.match(pattern, text):
                    numeric_value = float(re.sub(r'[^\d.-]', '', text))
                    if -20 <= numeric_value <= 20:
                        return numeric_value
            
            return None
            
        except Exception as e:
            print(f"Error scraping SG GDP growth: {e}")
            return None

    def scrape_sg_inflation_from_trading_economics(self):
        try:
            url = "https://tradingeconomics.com/singapore/inflation-cpi"
            response = requests.get(url, headers=self.headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            for element in soup.find_all(['span', 'div', 'td']):
                text = element.get_text(strip=True)
                pattern = r'^[+-]?\d+\.\d+%?$'
                if re.match(pattern, text):
                    numeric_value = float(re.sub(r'[^\d.-]', '', text))
                    if -10 <= numeric_value <= 20:
                        return numeric_value
            
            return None
            
        except Exception as e:
            print(f"Error scraping SG inflation: {e}")
            return None

    def scrape_sg_unemployment_rate(self):
        try:
            url = "https://tradingeconomics.com/singapore/unemployment-rate"
            response = requests.get(url, headers=self.headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            for element in soup.find_all(['span', 'div', 'td']):
                text = element.get_text(strip=True)
                pattern = r'^\d+\.\d+%?$'
                if re.match(pattern, text):
                    numeric_value = float(re.sub(r'[^\d.]', '', text))
                    if 0.5 <= numeric_value <= 10.0:
                        return numeric_value
            
            return None
            
        except Exception as e:
            print(f"Error scraping SG unemployment rate: {e}")
            return None

    def scrape_sg_exports_growth(self):
        try:
            url = "https://tradingeconomics.com/singapore/exports"
            response = requests.get(url, headers=self.headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            for element in soup.find_all(['span', 'div', 'td']):
                text = element.get_text(strip=True)
                pattern = r'^[+-]?\d+\.\d+%?$'
                if re.match(pattern, text):
                    numeric_value = float(re.sub(r'[^\d.-]', '', text))
                    if -50 <= numeric_value <= 50:
                        return numeric_value
            
            return None
            
        except Exception as e:
            print(f"Error scraping SG exports growth: {e}")
            return None

    def scrape_sg_industrial_production(self):
        try:
            url = "https://tradingeconomics.com/singapore/industrial-production"
            response = requests.get(url, headers=self.headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            for element in soup.find_all(['span', 'div', 'td']):
                text = element.get_text(strip=True)
                pattern = r'^[+-]?\d+\.\d+%?$'
                if re.match(pattern, text):
                    numeric_value = float(re.sub(r'[^\d.-]', '', text))
                    if -50 <= numeric_value <= 50:
                        return numeric_value
            
            return None
            
        except Exception as e:
            print(f"Error scraping SG industrial production: {e}")
            return None

    def scrape_us_core_inflation_yoy(self):
        try:
            url = "https://tradingeconomics.com/united-states/core-inflation-rate"
            response = requests.get(url, headers=self.headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            for element in soup.find_all(['span', 'div', 'td']):
                text = element.get_text(strip=True)
                if re.match(r'^\d\.\d+%?$', text):
                    numeric_value = float(re.sub(r'[^\d.]', '', text))
                    if 1 <= numeric_value <= 8:
                        return numeric_value
            
            return None
            
        except Exception as e:
            print(f"Error scraping US core inflation: {e}")
            return None

    def scrape_us_nonfarm_payrolls(self):
        if self.fred_api_key:
            fred_data = self.scrape_fred_data('PAYEMS')
            if fred_data and fred_data['value']:
                return fred_data['value']
        
        return None

    def scrape_fed_funds_rate(self):
        if self.fred_api_key:
            fred_data = self.scrape_fred_data('FEDFUNDS')
            if fred_data and fred_data['value']:
                return fred_data['value']
        
        return None

    def scrape_mas_policy_rate(self):
        try:
            return None
            
        except Exception as e:
            print(f"Error scraping MAS policy rate: {e}")
            return None

    def get_all_singapore_bond_yields(self):
        bond_data = {}
        
        bond_data['SG_1Y_yield'] = self.scrape_singapore_1y_bond()
        time.sleep(1)
        
        bond_data['SG_2Y_yield'] = self.scrape_singapore_2y_bond()
        time.sleep(1)
        
        bond_data['SG_5Y_yield'] = self.scrape_singapore_5y_bond()
        time.sleep(1)
        
        bond_data['SG_10Y_yield'] = self.scrape_singapore_10y_bond()
        time.sleep(1)
        
        return bond_data

    def get_all_us_treasury_yields(self):
        treasury_data = {}
        
        treasury_data['US_2Y_yield'] = self.scrape_us_2y_treasury()
        time.sleep(1)
        
        treasury_data['US_5Y_yield'] = self.scrape_us_5y_treasury()
        time.sleep(1)
        
        treasury_data['US_10Y_yield'] = self.scrape_us_10y_treasury()
        time.sleep(1)
        
        treasury_data['US_30Y_yield'] = self.scrape_us_30y_treasury()
        time.sleep(1)
        
        return treasury_data

    def get_all_market_indicators(self):
        indicators = {}
        
        indicators['VIX'] = self.scrape_vix()
        indicators['DXY'] = self.scrape_dxy()
        indicators['Gold'] = self.scrape_gold_price()
        indicators['Oil_WTI'] = self.scrape_oil_wti_price()
        indicators['MSCI_Asia_Pacific'] = self.scrape_msci_asia_pacific()
        
        indicators['daily_change_pct'] = self.scrape_sgd_usd_daily_change_pct()
        indicators['sp500_change_pct'] = self.scrape_sp500_change_pct()
        indicators['gold_change_pct'] = self.scrape_gold_change_pct()
        indicators['singapore_sti_change_pct'] = self.scrape_singapore_sti_change_pct()
        indicators['nikkei_change_pct'] = self.scrape_nikkei_change_pct()
        indicators['hang_seng_change_pct'] = self.scrape_hang_seng_change_pct()
        indicators['oil_wti_change_pct'] = self.scrape_oil_wti_change_pct()
        indicators['dbs_change_pct'] = self.scrape_dbs_change_pct()
        
        indicators['high_52w'] = self.scrape_sgd_usd_52w_high()
        indicators['low_52w'] = self.scrape_sgd_usd_52w_low()
        
        return indicators

    def get_all_fred_economic_data(self):
        if not self.fred_api_key:
            return {}
        
        fred_data = {}
        
        fred_series = {
            'US_GDP_GROWTH_YOY': 'A191RL1Q225SBEA',
            'US_INFLATION_YOY': 'FPCPITOTLZGUSA',
            'US_GDP': 'GDP',
            'US_CPI': 'CPIAUCSL',
            'US_UNEMPLOYMENT': 'UNRATE',
            'EFFECTIVE_FED_RATE': 'FEDFUNDS',
        }
        
        for name, series_id in fred_series.items():
            data = self.scrape_fred_data(series_id)
            if data:
                fred_data[name] = data['value']
                fred_data[f'{name}_date'] = data['date']
            else:
                fred_data[name] = None
        
        fred_data['SG_GDP_GROWTH_YOY'] = self.scrape_sg_gdp_from_trading_economics()
        fred_data['SG_INFLATION_YOY'] = self.scrape_sg_inflation_from_trading_economics()
        
        return fred_data

    def get_all_high_priority_data(self):
        print("collecting high priority indicators")
        all_data = {}
        
        all_data['weekly_change_pct'] = self.scrape_sgd_usd_weekly_change_pct()
        all_data['monthly_change_pct'] = self.scrape_sgd_usd_monthly_change_pct()
        
        all_data['volatility_20d'] = self.scrape_sgd_usd_volatility_20d()
        
        all_data['usd_5y_yield'] = self.scrape_us_5y_treasury()
        
        all_data['eur_usd_change_pct'] = self.scrape_eur_usd_change_pct()
        all_data['usd_jpy_change_pct'] = self.scrape_usd_jpy_change_pct()
        all_data['vix_change_pct'] = self.scrape_vix_change_pct()
        all_data['dxy_change_pct'] = self.scrape_dxy_change_pct()
        
        all_data['sg_unemployment_rate'] = self.scrape_sg_unemployment_rate()
        all_data['sg_exports_growth'] = self.scrape_sg_exports_growth()
        
        all_data['uob_change_pct'] = self.scrape_uob_change_pct()
        all_data['dbs_change_pct'] = self.scrape_dbs_change_pct()
        all_data['sg_reit_change_pct'] = self.scrape_sg_reit_change_pct()
        
        all_data['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        return all_data

    def get_all_medium_priority_data(self):
        print("collecting medium priority indicators")
        all_data = {}
        
        print("collecting market levels")
        all_data['sp500_level'] = self.scrape_sp500_level()
        all_data['vix_level'] = self.scrape_vix_level()
        all_data['dxy_level'] = self.scrape_dxy_level()
        all_data['nikkei_level'] = self.scrape_nikkei_level()
        all_data['hang_seng_level'] = self.scrape_hang_seng_level()
        all_data['singapore_sti_level'] = self.scrape_singapore_sti_level()
        all_data['gold_price'] = self.scrape_gold_price()
        all_data['oil_wti_price'] = self.scrape_oil_wti_price()
        
        print("collecting currency rates")
        all_data['eur_usd_rate'] = self.scrape_eur_usd_rate()
        all_data['usd_jpy_rate'] = self.scrape_usd_jpy_rate()
        
        print("collecting Singapore stock prices")
        all_data['dbs_stock_price'] = self.scrape_dbs_stock_price()
        all_data['uob_stock_price'] = self.scrape_uob_stock_price()
        all_data['sg_reit_index'] = self.scrape_sg_reit_index()
        
        print("collecting commodity indicators")
        all_data['baltic_dry_index'] = self.scrape_baltic_dry_index()
        all_data['palm_oil_price'] = self.scrape_palm_oil_price()
        all_data['baltic_dry_change_pct'] = self.scrape_baltic_dry_change_pct()
        all_data['palm_oil_change_pct'] = self.scrape_palm_oil_change_pct()
        
        print("collecting economic indicators")
        all_data['sg_industrial_production'] = self.scrape_sg_industrial_production()
        all_data['us_core_inflation_yoy'] = self.scrape_us_core_inflation_yoy()
        all_data['us_nonfarm_payrolls'] = self.scrape_us_nonfarm_payrolls()
        all_data['fed_funds_rate'] = self.scrape_fed_funds_rate()
        all_data['mas_policy_rate'] = self.scrape_mas_policy_rate()
        
        all_data['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        return all_data

    def collect_all_market_data(self):
        all_data = {}
        
        forex_data = self.get_comprehensive_forex_data()
        all_data.update(forex_data)
        
        sg_bonds = self.get_all_singapore_bond_yields()
        all_data.update(sg_bonds)
        
        us_bonds = self.get_all_us_treasury_yields()
        all_data.update(us_bonds)
        
        market_indicators = self.get_all_market_indicators()
        all_data.update(market_indicators)
        
        fred_data = self.get_all_fred_economic_data()
        all_data.update(fred_data)
        
        high_priority = self.get_all_high_priority_data()
        all_data.update(high_priority)
        
        medium_priority = self.get_all_medium_priority_data()
        all_data.update(medium_priority)
        
        all_data['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        return all_data

    def test_time_period_changes(self):
        
        weekly_result = self.scrape_sgd_usd_weekly_change_pct()

        monthly_result = self.scrape_sgd_usd_monthly_change_pct()

        success_count = sum([1 for x in [weekly_result, monthly_result] if x is not None])

        return weekly_result, monthly_result

    def test_marketwatch_scrapers(self):

        uob_change = self.scrape_uob_change_pct()
        print(f"UOB Change: {uob_change:+.2f}%" if uob_change is not None else "UOB Change: N/A")
        
        dbs_change = self.scrape_dbs_change_pct()
        print(f"DBS Change: {dbs_change:+.2f}%" if dbs_change is not None else "DBS Change: N/A")
        
        success_count = sum([1 for x in [uob_change, dbs_change] if x is not None])
        return uob_change, dbs_change

    def print_simple_results(self, data):
        sgd_usd = data.get('current_rate')
        if sgd_usd:
            usd_sgd = 1 / sgd_usd
            print(f"SGDUSD rate: {sgd_usd:.4f}")
            print(f"USDSGD rate: {usd_sgd:.4f}")
        else:
            print("SGDUSD rate: N/A")
            print("USDSGD rate: N/A")
        
        daily_change = data.get('daily_change_pct')
        if daily_change is not None:
            print(f"daily_change_pct: {daily_change:+.2f}")
        else:
            print("daily_change_pct: N/A")
        
        high_52w = data.get('high_52w')
        print(f"high_52w: {high_52w:.4f}" if high_52w else "high_52w: N/A")
            
        low_52w = data.get('low_52w')
        print(f"low_52w: {low_52w:.4f}" if low_52w else "low_52w: N/A")
        
        bond_mapping = {
            'SG_1Y_yield': 'SG1Y',
            'SG_2Y_yield': 'SG2Y',
            'SG_5Y_yield': 'SG5Y', 
            'SG_10Y_yield': 'SG10Y',
            'US_2Y_yield': 'US2Y',
            'US_5Y_yield': 'US5Y',
            'US_10Y_yield': 'US10Y',
            'US_30Y_yield': 'US30Y'
        }
        
        for key, label in bond_mapping.items():
            value = data.get(key)
            print(f"{label}: {value:.3f}" if value else f"{label}: N/A")
        
        indicator_mapping = {
            'VIX': 'VIX',
            'DXY': 'DXY',
            'Gold': 'Gold',
            'Oil_WTI': 'Oil',
            'MSCI_Asia_Pacific': 'MSCI_Asia'
        }
        
        for key, label in indicator_mapping.items():
            value = data.get(key)
            print(f"{label}: {value:.2f}" if value else f"{label}: N/A")
        
        change_mapping = {
            'sp500_change_pct': 'sp500_change_pct',
            'gold_change_pct': 'gold_change_pct',
            'singapore_sti_change_pct': 'singapore_sti_change_pct',
            'nikkei_change_pct': 'nikkei_change_pct',
            'hang_seng_change_pct': 'hang_seng_change_pct',
            'oil_wti_change_pct': 'oil_wti_change_pct',
            'dbs_change_pct': 'dbs_change_pct'
        }
        
        for key, label in change_mapping.items():
            value = data.get(key)
            print(f"{label}: {value:+.2f}" if value is not None else f"{label}: N/A")
        
        economic_mapping = {
            'SG_GDP_GROWTH_YOY': 'sg_gdp_growth_yoy',
            'US_GDP_GROWTH_YOY': 'us_gdp_growth_yoy',
            'SG_INFLATION_YOY': 'sg_inflation_yoy',
            'US_INFLATION_YOY': 'us_inflation_yoy'
        }
        
        for key, label in economic_mapping.items():
            value = data.get(key)
            print(f"{label}: {value:.2f}" if value is not None else f"{label}: N/A")

    def print_high_priority_results(self, data):

        weekly_change = data.get('weekly_change_pct')
        print(f"weekly_change_pct: {weekly_change:+.2f}" if weekly_change is not None else "weekly_change_pct: N/A")
        
        monthly_change = data.get('monthly_change_pct')
        print(f"monthly_change_pct: {monthly_change:+.2f}" if monthly_change is not None else "monthly_change_pct: N/A")
        
        usd_5y = data.get('usd_5y_yield')
        print(f"usd_5y_yield: {usd_5y:.3f}" if usd_5y is not None else "usd_5y_yield: N/A")
        
        forex_mapping = {
            'eur_usd_change_pct': 'eur_usd_change_pct',
            'usd_jpy_change_pct': 'usd_jpy_change_pct',
            'vix_change_pct': 'vix_change_pct',
            'dxy_change_pct': 'dxy_change_pct'
        }
        
        for key, label in forex_mapping.items():
            value = data.get(key)
            print(f"{label}: {value:+.2f}" if value is not None else f"{label}: N/A")
        
        sg_unemployment = data.get('sg_unemployment_rate')
        print(f"sg_unemployment_rate: {sg_unemployment:.2f}" if sg_unemployment is not None else "sg_unemployment_rate: N/A")
        
        sg_exports = data.get('sg_exports_growth')
        print(f"sg_exports_growth: {sg_exports:+.2f}" if sg_exports is not None else "sg_exports_growth: N/A")
        
        uob_change = data.get('uob_change_pct')
        print(f"uob_change_pct: {uob_change:+.2f}" if uob_change is not None else "uob_change_pct: N/A")
        
        dbs_change = data.get('dbs_change_pct')
        print(f"dbs_change_pct: {dbs_change:+.2f}" if dbs_change is not None else "dbs_change_pct: N/A")
        
        sg_reit = data.get('sg_reit_change_pct')
        print(f"sg_reit_change_pct: {sg_reit:+.2f}" if sg_reit is not None else "sg_reit_change_pct: N/A")

    def print_medium_priority_results(self, data):
        market_levels = {
            'sp500_level': 'sp500_level',
            'vix_level': 'vix_level',
            'dxy_level': 'dxy_level',
            'nikkei_level': 'nikkei_level',
            'hang_seng_level': 'hang_seng_level',
            'singapore_sti_level': 'singapore_sti_level',
            'gold_price': 'gold_price',
            'oil_wti_price': 'oil_wti_price'
        }
        
        for key, label in market_levels.items():
            value = data.get(key)
            print(f"{label}: {value:.2f}" if value is not None else f"{label}: N/A")
        
        currency_rates = {
            'eur_usd_rate': 'eur_usd_rate',
            'usd_jpy_rate': 'usd_jpy_rate'
        }
        
        for key, label in currency_rates.items():
            value = data.get(key)
            print(f"{label}: {value:.4f}" if value is not None else f"{label}: N/A")
        
        sg_stocks = {
            'dbs_stock_price': 'dbs_stock_price',
            'uob_stock_price': 'uob_stock_price',
            'sg_reit_index': 'sg_reit_index'
        }
        
        for key, label in sg_stocks.items():
            value = data.get(key)
            print(f"{label}: {value:.2f}" if value is not None else f"{label}: N/A")
        
        commodities = {
            'baltic_dry_index': 'baltic_dry_index',
            'palm_oil_price': 'palm_oil_price',
            'baltic_dry_change_pct': 'baltic_dry_change_pct',
            'palm_oil_change_pct': 'palm_oil_change_pct'
        }
        
        for key, label in commodities.items():
            value = data.get(key)
            if 'change_pct' in key:
                print(f"{label}: {value:+.2f}" if value is not None else f"{label}: N/A")
            else:
                print(f"{label}: {value:.2f}" if value is not None else f"{label}: N/A")
        
        economic = {
            'sg_industrial_production': 'sg_industrial_production',
            'us_core_inflation_yoy': 'us_core_inflation_yoy',
            'us_nonfarm_payrolls': 'us_nonfarm_payrolls',
            'fed_funds_rate': 'fed_funds_rate',
            'mas_policy_rate': 'mas_policy_rate'
        }
        
        for key, label in economic.items():
            value = data.get(key)
            if 'payrolls' in key:
                print(f"{label}: {value:.0f}" if value is not None else f"{label}: N/A")
            else:
                print(f"{label}: {value:.2f}" if value is not None else f"{label}: N/A")

    def print_comprehensive_results(self, data):
        print("\n" + "="*80)
        print("COMPREHENSIVE SGD/USD MARKET DATA")
        print("="*80)
        
        self.print_simple_results(data)
        
        self.print_high_priority_results(data)
        
        self.print_medium_priority_results(data)

        successful_scrapers = sum([1 for v in data.values() if v is not None and v != "N/A" and v != data.get('timestamp')])
        total_scrapers = len(data) - 1
        
        print(successful_scrapers)


if __name__ == "__main__":
    FRED_API_KEY = os.getenv('FRED_API_KEY')
    
    scrapers = ComprehensiveSGDUSDScrapers(fred_api_key=FRED_API_KEY)
    
    print("scraping")
    all_data = scrapers.collect_all_market_data()
    
    scrapers.print_comprehensive_results(all_data)

scraping
collecting high priority indicators
collecting medium priority indicators
collecting market levels
collecting currency rates
collecting Singapore stock prices
collecting commodity indicators
collecting economic indicators

COMPREHENSIVE SGD/USD MARKET DATA
SGDUSD rate: 0.7783
USDSGD rate: 1.2848
daily_change_pct: +0.08
high_52w: 0.7876
low_52w: 0.7061
SG1Y: 1.770
SG2Y: 1.671
SG5Y: 1.751
SG10Y: 2.086
US2Y: 3.863
US5Y: 3.948
US10Y: 4.421
US30Y: 4.995
VIX: 16.60
DXY: 98.20
Gold: 3358.45
Oil: 67.42
MSCI_Asia: 204.28
sp500_change_pct: -0.11
gold_change_pct: +0.40
singapore_sti_change_pct: +0.76
nikkei_change_pct: -0.21
hang_seng_change_pct: +1.33
oil_wti_change_pct: -0.27
dbs_change_pct: +0.22
sg_gdp_growth_yoy: 3.90
us_gdp_growth_yoy: -0.50
sg_inflation_yoy: 0.90
us_inflation_yoy: 2.95
weekly_change_pct: N/A
monthly_change_pct: +0.13
usd_5y_yield: 3.948
eur_usd_change_pct: +0.25
usd_jpy_change_pct: +0.11
vix_change_pct: +0.79
dxy_change_pct: -0.27
sg_unemployment_rate: 2.10
sg_exp